# Importing necessary libraries

In [ ]:
import os
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.results_plotter import ts2xy, load_results
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecMonitor,VecFrameStack
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor
from gym.wrappers import ResizeObservation, GrayScaleObservation
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import retro

# Define callback for saving the best model during training

In [ ]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq:
    :param log_dir: Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: Verbosity level.
    """
    def __init__(self, check_freq: int, log_dir: str, verbose: int = 1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model')
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), 'timesteps')
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose > 0:
                print(f"Num timesteps: {self.num_timesteps}")
                print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose > 0:
                    print(f"Saving new best model to {self.save_path}")
                  self.model.save(self.save_path)

# Set up directories for saving models and logs

In [ ]:
LOG_DIR = './logs08/'
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

# Setting the environment 

In [ ]:
def make_env(env_id):
    def _init():
        env = gym_super_mario_bros.make(env_id)
        env = JoypadSpace(env, SIMPLE_MOVEMENT)
        env = ResizeObservation(env, shape=(84, 84))
        env = GrayScaleObservation(env, keep_dim=True)
        env = MaxAndSkipEnv(env, 4)
        return env

    return _init


In [ ]:
if __name__ == '__main__':
    
    env_id = 'SuperMarioBros-1-2-v0'
    environment_count = 4
    
    
    env = VecMonitor(VecFrameStack(SubprocVecEnv([make_env(env_id) for i in range(environment_count)]), n_stack=4, channels_order='last'), LOG_DIR)


# Load pre-trained model

In [ ]:
model = PPO.load('./logs08/1620000v2', env=env)

In [ ]:
learning_rate=model.learning_rate
print(learning_rate)
model.learning_rate = 0.000005




# Or make a Model

In [ ]:
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log="./board/", learning_rate=0.00003)



# Train the model


In [ ]:
callback = SaveOnBestTrainingRewardCallback(check_freq=2500, log_dir=LOG_DIR) 

model.learn(total_timesteps=20000000, callback=callback, tb_log_name="PPO-00003")
